In [4]:
#!/.conda/envs/dp python
# -*- coding: utf-8 -*-

"""
图像转为8位uint格式
~~~~~~~~~~~~~~~~
code by wHy
Aerospace Information Research Institute, Chinese Academy of Sciences
wanghaoyu191@mails.ucas.ac.cn
"""
from pathlib import Path
from osgeo import gdal,ogr,osr
import os
import sys
import math
from osgeo.ogr import Geometry, Layer
from tqdm import tqdm
import numpy as np
import fnmatch
import copy

In [5]:
def write_img(out_path, im_proj, im_geotrans, im_data):
    """output img

    Args:
        out_path: Output path
        im_proj: Affine transformation parameters
        im_geotrans: spatial reference
        im_data: Output image data

    """
    # identify data type 
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    # calculate number of bands
    if len(im_data.shape) > 2:  
        im_bands, im_height, im_width = im_data.shape
    else:  
        im_bands, (im_height, im_width) = 1, im_data.shape

    # create new img
    driver = gdal.GetDriverByName("GTiff")
    new_dataset = driver.Create(
        out_path, im_width, im_height, im_bands, datatype)
    new_dataset.SetGeoTransform(im_geotrans)
    new_dataset.SetProjection(im_proj)
    if im_bands == 1:
        new_dataset.GetRasterBand(1).WriteArray(im_data)
    else:
        for i in range(im_bands):
            new_dataset.GetRasterBand(i + 1).WriteArray(im_data[i])

    del new_dataset

def read_img(sr_img):
    """read img

    Args:
        sr_img: The full path of the original image

    """
    im_dataset = gdal.Open(sr_img)
    if im_dataset == None:
        print('open sr_img false')
        sys.exit(1)
    im_geotrans = im_dataset.GetGeoTransform()
    im_proj = im_dataset.GetProjection()
    im_width = im_dataset.RasterXSize
    im_height = im_dataset.RasterYSize
    im_data = im_dataset.ReadAsArray(0, 0, im_width, im_height)

    del im_dataset

    return im_data, im_proj, im_geotrans

In [6]:

input_folder = r'D:\BaiduNetdiskDownload\MHdataset\MHparcel\hetian-GF2\1-tif_field'  # 输入文件夹路径
output_folder = r'D:\BaiduNetdiskDownload\MHdataset\MHparcel\hetian-GF2\image-uint8'  # 输出文件夹路径

img_type = '*.tif'

# 确保输出文件夹存在
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

listpic = fnmatch.filter(os.listdir(input_folder), img_type)

'''逐个读取影像'''
for img in listpic:
    img_full_path = input_folder + '/' + img
    data, proj_temp, geotrans_temp = read_img(img_full_path)
    img_shape = data.shape
    if len(img_shape) == 2:
        output_data = np.zeros((img_shape[0], img_shape[1]), dtype=np.uint8)
        min_value = np.min(data)
        max_value = np.max(data)
        print(min_value, max_value) # 注意最小值和最大值
        output_data = ((data - min_value) / (max_value - min_value) * 255).astype(np.uint8)
    else:
        output_data = np.zeros((img_shape[0], img_shape[1], img_shape[2]), dtype=np.uint8)
        for i in range(img_shape[0]): # 读取每个波段
            data_temp = data[i, :, :]
            # 对像素值进行缩放和截断操作
            data_temp = np.nan_to_num(data_temp, nan=0) # 处理nan
            min_value = np.min(data_temp)
            max_value = np.max(data_temp)
            print(min_value, max_value) # 注意最小值和最大值
            output_array = ((data_temp - min_value) / (max_value - min_value) * 255).astype(np.uint8)
            output_data[i, :, :] = output_array
    
    output_full_path = output_folder + '/' + img
    write_img(output_full_path, proj_temp, geotrans_temp, output_data)

c:\Users\ASUS\anaconda3\envs\dp\lib\site-packages\osgeo\gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


18.622005 61.346394
15.0741625 63.567364
10.6717205 63.23262
7.8648963 81.89104
25.844156 141.78987
22.384594 129.74905
15.630068 108.8406
16.5839 110.49706
40.92015 86.85165
37.66699 87.32564
31.067467 83.908646
24.868578 64.05289
27.066393 102.011
23.61034 96.65796
17.189697 89.05634
17.510418 94.73858
25.240334 61.69111
21.221 60.668667
15.7377205 56.45647
10.757468 61.1226
24.858744 89.79666
21.368711 88.75743
15.607564 84.58536
18.516672 89.80734
33.56992 94.29924
27.081356 91.875946
19.07428 85.67159
21.930933 101.86066
28.71567 96.71297
22.260483 94.106445
15.371056 88.46316
17.192707 82.262726
25.427425 136.02777
20.092876 117.400894
13.670814 92.89893
16.4013 105.78442
0.0 91.3936
0.0 88.78474
0.0 82.372765
0.0 84.78131
23.495535 93.12421
20.370926 90.47015
14.32319 86.40629
16.961357 84.730865
22.670963 91.32626
19.285456 88.48686
13.318073 82.03302
15.915864 86.06902
25.06329 127.363014
15.377044 107.48432
9.805208 100.71739
8.351035 105.11998
26.654692 128.5115
16.556273 12